In this notebook, the identified spectra for the files used in the protein expression atlas are counted.

In [1]:
import subprocess
import mysql.connector
import pandas as pd
from pathlib import Path
import glob

In [8]:
PXD_files = ['PXD000533',
 'PXD004280',
 'PXD002842',
 'PXD008996',
 'PXD006035',
 'PXD008719',
 'PXD006591',
 'PXD003406',
 'PXD003407',
 'PXD001327',
 'PXD002057',
 'PXD004824',
 'PXD003438',
 'PXD001352',
 'PXD000661',
 'PXD008381',
 'PXD009149',
 'PXD005045',
 'PXD000529',
 'PXD000443',
 'PXD010538',
 'PXD009442',
 'PXD009560',
 'PXD003252',
 'PXD007543',
 'PXD008693',
 'PXD000426',
 'PXD007759',
 'PXD018066',
 'PXD017391',
 'PXD016742']

In [9]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',database='expression_atlas_cells')
mycursor = conn.cursor(buffered = True)

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")

connection succesfull


In [11]:
NSAF_atlas = pd.read_csv("/home/compomics/Sam/git/python/master_thesis/Atlas_creation/proteome_nsaf_1.csv")
NSAF_atlas = NSAF_atlas.pivot(index = "assay_id", columns = "uniprot_id", values = "NSAF").fillna(0).reset_index()
NSAF_atlas.assay_id

0      30960
1      30961
2      30962
3      30963
4      30964
       ...  
854    32907
855    32908
856    32909
857    32910
858    32911
Name: assay_id, Length: 859, dtype: int64

In [18]:
df_sql = pd.read_sql("SELECT assay_id, filename FROM assay", conn)
merged_df = pd.merge(NSAF_atlas, df_sql, on = "assay_id")
merged_df["assay_id filename".split()]

merged_df["filename"] = merged_df.filename.apply(lambda x: x + '.mgf')
merged_df = merged_df["assay_id filename".split()]
merged_df

,assay_id,filename
0,30960,3B10-1.mgf
1,30961,3B10-2.mgf
2,30962,3B1-1.mgf
3,30963,3B11-1.mgf
4,30964,3B11-2.mgf
...,...,...
854,32907,20190828_Lumos_ML_NP_EXP1_M2_48h_Rep3.mgf
855,32908,20190828_Lumos_ML_NP_EXP1_ND_Rep1.mgf
856,32909,20190828_Lumos_ML_NP_EXP1_ND_Rep2.mgf
857,32910,20190828_Lumos_ML_NP_EXP1_ND_Rep3.mgf


In [14]:
filenames[:5]

['3B10-1', '3B10-2', '3B1-1', '3B11-1', '3B11-2']

In [11]:
for f in PXD_files:
    if os.path.isdir('/home/compomics/mounts/conode53/pride/PRIDE_DATA/' + f + '/MGF'):
        conode = 'conode53/pride'
    elif os.path.isdir('/home/compomics/mounts/conode53/pride2/PRIDE_DATA/' + f + '/MGF'):
        conode = 'conode53/pride2'
    elif os.path.isdir('/home/compomics/mounts/conode54/pride/PRIDE_DATA/' + f + '/MGF'):
        conode = 'conode54/pride'
    elif os.path.isdir('/home/compomics/mounts/conode54/pride2/PRIDE_DATA/' + f + '/MGF'):
        conode = 'conode54/pride2'
    elif os.path.isdir('/home/compomics/mounts/conode55/pride/PRIDE_DATA/' + f + '/MGF'):
        conode = '55'
    
    path = '/home/compomics/mounts/'+ conode +'/PRIDE_DATA/' + f + '/MGF/*'
    print(f'found project {f}')
    copy_command = "cp -a " + path + ' /home/compomics/Sam/git/python/'
    subprocess.call(copy_command, shell=True)
    print(f'copied project {f}')
    current_dir = "/home/compomics/Sam/git/python/"
    count_command = 'cd ' + current_dir + "; /home/compomics/Sam/git/python/count_spectra.sh"
    subprocess.call(count_command, shell=True)
    print(f'counted project {f}')
print('finished')

found project PXD000533
copied project PXD000533
counted project PXD000533
found project PXD004280
copied project PXD004280
counted project PXD004280
found project PXD002842
copied project PXD002842
counted project PXD002842
found project PXD008996
copied project PXD008996
counted project PXD008996
found project PXD006035
copied project PXD006035
counted project PXD006035
found project PXD008719
copied project PXD008719
counted project PXD008719
found project PXD006591
copied project PXD006591
counted project PXD006591
found project PXD003406
copied project PXD003406
counted project PXD003406
found project PXD003407
copied project PXD003407
counted project PXD003407
found project PXD001327
copied project PXD001327
counted project PXD001327
found project PXD002057
copied project PXD002057
counted project PXD002057
found project PXD004824
copied project PXD004824
counted project PXD004824
found project PXD003438
copied project PXD003438
counted project PXD003438
found project PXD001352
c

cp: cannot stat '/home/compomics/mounts/55/PRIDE_DATA/PXD018066/MGF/*': No such file or directory
cp: cannot stat '/home/compomics/mounts/55/PRIDE_DATA/PXD017391/MGF/*': No such file or directory
cp: cannot stat '/home/compomics/mounts/55/PRIDE_DATA/PXD016742/MGF/*': No such file or directory


In [23]:
df = pd.read_csv("output.txt", sep =":", header = None, names = ["filename", "spectra"])
df.shape

(1483, 2)

In [32]:
output_file_merge = pd.merge(merged_df, df, on = "filename", how="inner")
output_file_merge = output_file_merge.drop_duplicates()
output_file_merge.shape

(837, 3)

In [33]:
output_file_merge.spectra.sum()

28024239

Bash script:
<br>
#! /bin/sh <br>
gzip -dk -S .gzip -r * <br>
grep -r --include \*.mgf -c "^BEGIN IONS"  >output.txt <br>
find . -name "*.mgf" -type f -delete <br>